In [7]:
import pandas as pd

In [8]:
#from google.colab import drive

#drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
#df_1 = pd.read_csv('/content/drive/MyDrive/human_vs_twitter/full_dataset/goemotions_1.csv')

**Importing and renaming columns of abbreviation and slang datasets:**

In [10]:
df_abbr = pd.read_csv('../data/raw/Abbreviations_and_Slang.csv')

In [11]:
df_slang = pd.read_csv('../data/raw/slang.csv')[['acronym', 'expansion']]
df_slang.rename(columns = {'acronym':'Abbreviations','expansion':'Text'}, inplace = True)

In [12]:
df_slang

,Abbreviations,Text
0,2day,today
1,2m2h,too much too handle
2,2moro,tomorrow
3,2nite,tonight
4,4eae,for ever and ever
...,...,...
3352,yw,you are welcome
3353,ywca,young womens christian association
3354,ywimc,your wish is my command
3355,ywsyls,you win some you lose some


**Both slang and abbreviation datasets together:**

In [13]:
df_slang_all = pd.concat([df_abbr , df_slang], axis=0)
df_slang_all

,Abbreviations,Text
0,lol,laughing
1,rofl,laughing
2,brb,be right back
3,ily,i love you
4,ty,thank you
...,...,...
3352,yw,you are welcome
3353,ywca,young womens christian association
3354,ywimc,your wish is my command
3355,ywsyls,you win some you lose some


**Checking new DF for duplicates and possible null values:**

In [14]:
df_slang_all.drop_duplicates(inplace=True)
df_slang_all

,Abbreviations,Text
0,lol,laughing
1,rofl,laughing
2,brb,be right back
3,ily,i love you
4,ty,thank you
...,...,...
3350,yum,said when something tastes good or is attractive
3351,yvw,you are very welcome
3353,ywca,young womens christian association
3354,ywimc,your wish is my command


In [15]:
df_slang_all.isnull().sum()

Abbreviations    0
Text             0
dtype: int64

**Transforming DF into dict for mapping:**

In [16]:
slang_dict = dict(zip(df_slang_all.Abbreviations, df_slang_all.Text))
slang_dict

{'lol': 'laughing out loud',
 'rofl': 'rolling on floor laughing',
 'brb': 'be right back',
 'ily': 'i love you',
 'ty': 'thank you',
 'imy': 'i miss you',
 'yolo': 'you only live once',
 'fomo': 'fear of missing out',
 'idk': 'i do not know',
 'idc': 'i do not care',
 'ffs': 'for freaks sake',
 'smh': 'shaking my head',
 'ngl': 'not going to lie',
 'w': 'with',
 'abt': 'about',
 'u': 'you only live once',
 'r': 'are',
 'gtg': 'got to go',
 'nvm': 'never mind',
 'bcoz': 'because',
 'coz': 'because',
 'bcos': 'because',
 'cld': 'could',
 'ez': 'easy',
 'fbm': 'fine by me',
 'ftw': 'for the win',
 'fyi': 'for your information',
 'ik': 'i know',
 'wfh': 'work from home',
 'lmfao': 'laughing my freaking ass off',
 'lmk': 'let me know',
 'af': 'as freak',
 'aight': 'alright',
 'awol': 'absent without leave',
 'irl ': 'in real life',
 'bt': 'but',
 'bb': 'big brother be back',
 'btw': 'by the way',
 'cu': 'see you',
 'idgaf': "i don't give a freak",
 'dgaf': "don't give a freak",
 'df': 'the

**Mapping dict to the original text:**

In [18]:
import string

class SlangTranslation:

  def __init__(self, col):

    self.col = col

  def remove_punctuation(self, txt):
    """Iterates through each word of the string and removes punctuation"""

    for punctuation in string.punctuation:
        txt = txt.replace(punctuation, ' ')

    return txt

  def string_translator(self, txt):
    """Iterates through each word of the string and translates them"""

    txt = ' '.join([slang_dict.get(i, i) for i in txt.lower().split()])

    return txt

  def apply_translator(self):
    """Takes the text column as input, outputs the same column translated."""

    txt = self.col.apply(self.remove_punctuation)

    txt = txt.apply(self.string_translator)

    return txt

In [19]:
SlangTranslation(test['string']).apply_translator()

0                cutie you bread car oh my god
1    test this phrase oh my god what you doing
Name: string, dtype: object

In [20]:
SlangTranslation(df_1['text']).apply_translator()

0                                           that game hurt
1        sexuality shouldn’t be a grouping category inf...
2              you do right if you don t care then fuck em
3                                        man i love reddit
4        name wait and see nowhere near them happy endi...
                               ...                        
69995    information technology s about fucking tears i...
69996    this is great can anyone make a request where ...
69997    i’m sorry can you please explain what are the ...
69998              no but information technology should be
69999    this is significant other shout out cruel i li...
Name: text, Length: 70000, dtype: object

**Preprocessing:**

In [21]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.9 MB/s eta 0:00:00


In [22]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**’**

In [68]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import string
import unidecode

class PreprocessingText:
    def __init__(self, col):
        self.col = col
    def cleaning_text(self, txt):
        """
        Transform everything in lowercase, strip the text and remove everything that's not letters or space
        """
        txt = txt.lower()
        txt = txt.replace('’', '').strip()
        text_cleaned = ''.join(filter(lambda x: x.isalpha() or x.isspace(), txt)) # Remove everything that's not letters or space


        return text_cleaned
    def tokenizing_text(self, txt):
        """
        Create stopword list, tokenize the words and return the text tokenized without stopwords
        """
        stop_words = set(stopwords.words('english')) # Create stopword list
        tokenized = word_tokenize(txt) # Tokenize
        tokenized_text = [word for word in tokenized if not word in stop_words] # Tokenizing text
        return tokenized_text
    def lemmatizing_text(self, tokenized_txt):
        """
        Lemmatize the text and return a cleaned sentence
        """
        lemmatized = [
            WordNetLemmatizer().lemmatize(word, pos='v') for word in tokenized_txt
        ] # Lematize
        cleaned_sentence = " ".join(word for word in lemmatized) # Lemmatized text
        return cleaned_sentence
    def preprocessing(self):
        """
        """
        txt = self.col
        txt = txt.apply(self.cleaning_text)
        txt = txt.apply(self.tokenizing_text)
        txt = txt.apply(self.lemmatizing_text)
        return txt

**Importing all dataframes and concatenating:**

In [24]:
#df_1 = pd.read_csv('/content/drive/MyDrive/human_vs_twitter/full_dataset/goemotions_1.csv')
#df_2 = pd.read_csv('/content/drive/MyDrive/human_vs_twitter/full_dataset/goemotions_2.csv')
#df_3 = pd.read_csv('/content/drive/MyDrive/human_vs_twitter/full_dataset/goemotions_3.csv')

In [25]:
#df_all_raw = pd.concat([df_1, df_2, df_3])
#df_all_raw

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71220,Everyone likes [NAME].,ee6pagw,Senshado,heroesofthestorm,t3_agjf24,t3_agjf24,1.547634e+09,16,False,0,...,1,0,0,0,0,0,0,0,0,0
71221,Well when you’ve imported about a gazillion of...,ef28nod,5inchloser,nottheonion,t3_ak26t3,t3_ak26t3,1.548553e+09,15,False,0,...,0,0,0,0,0,0,0,0,0,0
71222,That looks amazing,ee8hse1,springt1me,shittyfoodporn,t3_agrnqb,t3_agrnqb,1.547684e+09,70,False,1,...,0,0,0,0,0,0,0,0,0,0
71223,The FDA has plenty to criticize. But like here...,edrhoxh,enamedata,medicine,t3_aejqzd,t1_edrgdtx,1.547169e+09,4,False,0,...,0,0,0,0,0,0,0,0,0,0


**Removing all rows with unclear definitions as well as unnecessary columns:**

In [26]:
#df_all_raw = df_all_raw[df_all_raw['example_very_unclear'] == False]

In [27]:
#df_all_raw.drop(['id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear'], axis = 1, inplace = True)

<ipython-input-27-026a03a08a33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_raw.drop(['id', 'author', 'subreddit', 'link_id', 'parent_id',


In [29]:
#df_all_raw.to_csv('/content/drive/My Drive/human_vs_twitter/GoEmotions_full_raw.csv', index=False)

**Slang translation on the raw dataset:**

In [30]:
df_preprocessed = pd.read_csv('../data/raw/GoEmotions_full_raw.csv')

df_preprocessed['text'] = SlangTranslation(df_all_raw['text']).apply_translator()

<ipython-input-30-028ff5446f32>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_preprocessed['text'] = SlangTranslation(df_all_raw['text']).apply_translator()


**Preprocessing whole dataset:**

In [32]:
df_preprocessed['text'] = PreprocessingText(df_preprocessed['text']).preprocessing()

<ipython-input-32-108564a531b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_preprocessed['text'] = PreprocessingText(df_preprocessed['text']).preprocessing()


In [33]:
df_preprocessed

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,game hurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,right care fuck em,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,man love reddit,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,name wait see nowhere near happy end high expl...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,right consider author note important document ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71219,well glad awful act make think healthy boundar...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71220,everyone like name,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
71221,well youve import gazillion country get serious,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71222,look amaze,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
import numpy as np

y = df_preprocessed.drop('text', axis = 1)

np.floor((np.floor(y.sum())*100)/np.floor(len(y)))

admiration         8.0
amusement          4.0
anger              3.0
annoyance          6.0
approval           8.0
caring             2.0
confusion          3.0
curiosity          4.0
desire             1.0
disappointment     4.0
disapproval        5.0
disgust            2.0
embarrassment      1.0
excitement         2.0
fear               1.0
gratitude          5.0
grief              0.0
joy                3.0
love               3.0
nervousness        0.0
optimism           4.0
pride              0.0
realization        4.0
relief             0.0
remorse            1.0
sadness            3.0
surprise           2.0
neutral           26.0
dtype: float64

**Exporting preprocessed data as csv:**

In [35]:
df_preprocessed.to_csv('/content/drive/MyDrive/human_vs_twitter/data_preprocessed.csv')

In [36]:
import pandas as pd

df_preprocessed = pd.read_csv('/content/drive/MyDrive/human_vs_twitter/data_preprocessed.csv').drop('Unnamed: 0', axis = 1)

In [37]:
df_preprocessed

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,game hurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,right care fuck em,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,man love reddit,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,name wait see nowhere near happy end high expl...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,right consider author note important document ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207809,well glad awful act make think healthy boundar...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207810,everyone like name,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
207811,well youve import gazillion country get serious,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207812,look amaze,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
df_preprocessed = df_preprocessed.dropna()
len(df_preprocessed)

207512

**Word2Vec**

In [86]:
vocab = df_preprocessed['text'].str.split().apply(pd.Series).stack().unique()

In [87]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GRU
from tensorflow.keras.optimizers import Adam

In [94]:
from gensim.models import Word2Vec
import tensorflow as tf

embedding_dim = 128
vocab_size = len(vocab)

word2vec = Word2Vec(vocab)
# word2vec.compile(optimizer='adam',
#                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
#                  metrics=['accuracy'])

In [ ]:
# Set parameters
max_features = 1 # Maximum number of words to get out of the data
max_len =  182 # Maximum feature length
#embedding_dim =   # Dimensionality of word embeddings
loss = 'categorical_crossentropy' # Loss function
optimizer= Adam()  # Optimizer
metrics= ['accuracy']  # Defining metric to accuracy

In [46]:
from sklearn.model_selection import train_test_split

X = df_preprocessed['text']

y = df_preprocessed.drop('text', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df_preprocessed['text'].str.len().max()

In [95]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
    return np.array(embedded_sentence)
# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
    return embed

In [97]:
from gensim.models import Word2Vec
import gensim.downloader

word_model = gensim.downloader.load('glove-wiki-gigaword-50')

# sentence = list(df_preprocessed['text'][78])

# embed = embedding(word_model, sentence)

[==================================================] 100.0% 66.0/66.0MB downloaded


In [98]:
X_train_embedded = embedding(word_model, X_train)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_pad = pad_sequences(X_train_embedded, dtype='float32', padding='pre')

In [ ]:
def init_model_lstm():
    """Function to create model architecture"""
    model = Sequential() # Instanciate the model
    # model.add(Embedding(max_features, input_length=max_len, output_dim = 28)) # Embedding
    model.add(LSTM(units=64, activation='tanh', input_shape = (182, 50))) # Input layer
    model.add(Dense(units=32, activation='tanh')) # Dense layer
    model.add(Dense(units=28, activation='softmax')) # Output layer
    return model

In [ ]:
def compile_model(model):
    """Function to compile the model"""
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=metrics
    )
    return model